# Electrical Energy Billing

In [1]:
import pandas as pd

tab3 = pd.read_csv("tab3_2025.csv", sep=";", decimal=',') # fattori di perdita

tab_oneri_di_rete = pd.read_csv("tab4_2025.csv", sep="\t", decimal=',') # tariffe di rete
comp_s1 = tab_oneri_di_rete['componente s1']
comp_s2 = tab_oneri_di_rete['componente s2']  
comp_s3 = tab_oneri_di_rete['componente s3']

tab_class0 = pd.read_csv("Classe0_minmax.csv", sep=";", decimal=',')
tab_ASOS1 = pd.read_csv("ASOS1_minmax.csv", sep=";", decimal=',')
tab_ASOS2 = pd.read_csv("ASOS2_minmax.csv", sep=";", decimal=',')
tab_ASOS3 = pd.read_csv("ASOS3_minmax.csv", sep=";", decimal=',')
tab_VAL= pd.read_csv("VAL_minmax.csv", sep=";", decimal=',')

tab_ARIM = pd.read_csv("ARIM_minmax.csv", sep=";", decimal=',')
tab_UC3 = pd.read_csv("UC3.csv", sep="\t", decimal=',')
tab_UC6 = pd.read_csv("UC6.csv", sep="\t", decimal=',')

tab_injected_cost = pd.read_csv("Corrispettivi_per_immissioni_energia_reattiva.csv", sep=";", decimal=',') # euro/kvarh
tab_withdrawn_cost = pd.read_csv("Corrispettivi_per_prelievi_energia_reattiva.csv", sep=";", decimal=',') # euro/kvarh


## input:
- type of consumer
- power of contract (kW)
- consumption (kWh)


In [2]:
# Mapping tra ConsumerType e "Tipologie di contratto" in tab3
def get_consumer_contracted(tab3, consumer_type, contracted_power_kW: float, voltage_kv: float=None):

    df = tab3[tab3["Tipo di Consumatore"] == consumer_type]
    if voltage_kv is not None:
        df = df[
            (df["Tensione Min di Fornitura (kV)"] <= voltage_kv) &
            (df["Tensione Max di Fornitura (kV)"] >= voltage_kv)
        ]
    else:
        df = df[
            (df["Potenza Min di Contratto (kW)"] <= contracted_power_kW) &
            (df["Potenza Max di Contratto (kW)"] >= contracted_power_kW)
        ]

    if df.empty:
        raise ValueError("Nessuna tariffa trovata per i parametri forniti.")
   
    return df.iloc[0]


In [3]:
# Mapping tra ConsumerType e "Tipologie di contratto" in oneri
import pandas as pd
from common.enums import ConsumerType


def get_oneri_di_sistema(consumer_type, contracted_power_kW: float, voltage_kv: float=None):
    if consumer_type == 0 and contracted_power_kW <= 3: # BT domestico residente
        tab = tab_class0
    elif consumer_type == 1 and contracted_power_kW <= 3: # BT domestico non residente
        tab = tab_ASOS1
    elif consumer_type in [2, 3]: # BT non domestiche, illuminazione pubblica, ricarica EV
        tab = tab_ASOS2
    elif consumer_type == 4 and 0 <= contracted_power_kW <= 10: 
        tab = tab_ASOS2
    elif consumer_type == 5 and contracted_power_kW <= 100:
        tab = tab_ASOS2
    elif consumer_type in [6,7] and contracted_power_kW > 500: # Media/Alta tensione
        tab = tab_ASOS3
    elif consumer_type == 8 and voltage_kv is not None and voltage_kv < 380:
        tab = tab_ASOS3
    elif consumer_type == 8 and voltage_kv is not None and voltage_kv >= 380: # Altissima tensione >=380 kV
        tab = tab_VAL
    else:
        raise ValueError(f"Tipo di consumatore {consumer_type} non valido o intervallo di potenza non previsto.")
    return tab

In [4]:
def calculate_electricity_bill_system_costs(
    consumption_kwh,  
    contracted_power_kW: float,
    consumer_type: int,
    tab3: pd.DataFrame,
    comp_s1, comp_s2, comp_s3,
    voltage_kv: float = None
):
    consumer_enum = ConsumerType(consumer_type)
    per_unit_costs, per_kw_costs, per_kwh_costs = 0, 0, 0
    
    if consumer_enum in [0, 1]:
        per_unit_costs += comp_s1 / 100
        per_kw_costs += (comp_s2 / 100)* contracted_power_kW
        per_kwh_costs += (comp_s3 / 100) * consumption_kwh
    else: # NO_DOMESTIC
        row = get_consumer_contracted(tab3, consumer_enum, contracted_power_kW, voltage_kv)
        per_unit_costs += (row['Quota fissa'] / 100) if not pd.isna(row['Quota fissa']) else 0
        per_kw_costs += (row['Quota potenza'] / 100) * contracted_power_kW if not pd.isna(row['Quota potenza']) else 0
        per_kwh_costs += (row['Quota energia'] / 100) * consumption_kwh if not pd.isna(row['Quota energia']) else 0
    
    row_oneri_sistema = get_oneri_di_sistema(consumer_enum, contracted_power_kW, voltage_kv)
    per_unit_costs += (row_oneri_sistema['Quota fissa'] / 100) if not pd.isna(row_oneri_sistema['Quota fissa']) else 0
    per_kw_costs += (row_oneri_sistema['Quota potenza'] / 100) * contracted_power_kW if not pd.isna(row_oneri_sistema['Quota potenza']) else 0
    per_kwh_costs += (row_oneri_sistema['Quota energia'] / 100) * consumption_kwh if not pd.isna(row_oneri_sistema['Quota energia']) else 0

    return per_unit_costs, per_kw_costs, per_kwh_costs


# Using the class ElectricityBillSystemCosts

In [5]:
def filter_tab_by_consumer_power(tab, consumer_type, contracted_power_kW=None, voltage_kv=None):
    
    df = tab[tab["Tipo di Consumatore"] == consumer_type]
    if df.empty:
        raise ValueError("Nessuna tariffa trovata per il tipo di consumatore")

    if contracted_power_kW is not None:
        power_mask = (
            ((df["Potenza Min di Contratto (kW)"].isna()) & (df["Potenza Max di Contratto (kW)"].isna())) |
            ((~df["Potenza Min di Contratto (kW)"].isna()) & df["Potenza Max di Contratto (kW)"].isna() & (contracted_power_kW >= df["Potenza Min di Contratto (kW)"])) |
            ((~df["Potenza Min di Contratto (kW)"].isna()) & (~df["Potenza Max di Contratto (kW)"].isna()) & (contracted_power_kW >= df["Potenza Min di Contratto (kW)"]) & (contracted_power_kW <= df["Potenza Max di Contratto (kW)"])) |
            ((df["Potenza Min di Contratto (kW)"].isna()) & (~df["Potenza Max di Contratto (kW)"].isna()) & (contracted_power_kW <= df["Potenza Max di Contratto (kW)"]))
        )
    else:
        power_mask = pd.Series(True, index=df.index)

    if voltage_kv is not None:
        voltage_mask = (
            ((df["Tensione Min di Fornitura (kV)"].isna()) & (df["Tensione Max di Fornitura (kV)"].isna())) |
            ((~df["Tensione Min di Fornitura (kV)"].isna()) & df["Tensione Max di Fornitura (kV)"].isna() & (voltage_kv >= df["Tensione Min di Fornitura (kV)"])) |
            ((~df["Tensione Min di Fornitura (kV)"].isna()) & (~df["Tensione Max di Fornitura (kV)"].isna()) & (voltage_kv >= df["Tensione Min di Fornitura (kV)"]) & (voltage_kv <= df["Tensione Max di Fornitura (kV)"])) |
            ((df["Tensione Min di Fornitura (kV)"].isna()) & (~df["Tensione Max di Fornitura (kV)"].isna()) & (voltage_kv <= df["Tensione Max di Fornitura (kV)"]))
        )
    else:
        voltage_mask = pd.Series(True, index=df.index)

    filtered_df = df[power_mask & voltage_mask]

    if filtered_df.empty:
        raise ValueError("Nessuna tariffa trovata per i parametri forniti")
    if contracted_power_kW is not None and filtered_df["Potenza Min di Contratto (kW)"].notna().any():
        selected_idx = filtered_df["Potenza Min di Contratto (kW)"].idxmax()
    elif voltage_kv is not None and filtered_df["Tensione Min di Fornitura (kV)"].notna().any():
        selected_idx = filtered_df["Tensione Min di Fornitura (kV)"].idxmax()
    else:
        selected_idx = filtered_df.index[0]

    selected_row = filtered_df.loc[selected_idx]

    return selected_row


In [6]:
def get_quote(tab, consumer_type, contracted_power_kW=None, voltage_kv=None):
    selected_row = filter_tab_by_consumer_power(tab, consumer_type, contracted_power_kW, voltage_kv)

    quota_fissa = selected_row.get("Quota fissa", 0)
    quota_potenza = selected_row.get("Quota potenza", 0)
    quota_energia = selected_row.get("Quota energia", 0)
    return quota_fissa, quota_potenza, quota_energia


In [7]:
def get_withdrawn_reactive_cost(tab, consumer_type, contracted_power_kW=None, voltage_kv=None):
    selected_row = filter_tab_by_consumer_power(tab, consumer_type, contracted_power_kW, voltage_kv)
    
    withdrawn_cost_f1 = selected_row.get("Prelievi Energia Reattiva F1", 0)
    withdrawn_cost_f2 = selected_row.get("Prelievi Energia Reattiva F2", 0)
    withdrawn_cost_f3 = selected_row.get("Prelievi Energia Reattiva F3", 0)
    return [withdrawn_cost_f1, withdrawn_cost_f2, withdrawn_cost_f3]


def get_injected_reactive_cost(tab, consumer_type, contracted_power_kW=None, voltage_kv=None):
    selected_row = filter_tab_by_consumer_power(tab, consumer_type, contracted_power_kW, voltage_kv)
    
    injected_cost_f1 = selected_row.get("Immissioni Energia Reattiva F1", 0)
    injected_cost_f2 = selected_row.get("Immissioni Energia Reattiva F2", 0)
    injected_cost_f3 = selected_row.get("Immissioni Energia Reattiva F3", 0)
    return [injected_cost_f1, injected_cost_f2, injected_cost_f3]

def get_reactive_cost(consumer_type, contracted_power_kW, voltage_kv):
    withdrawn_costs = get_withdrawn_reactive_cost(
        tab_withdrawn_cost, consumer_type, contracted_power_kW, voltage_kv
    )
    injected_costs = get_injected_reactive_cost(
        tab_injected_cost, consumer_type, contracted_power_kW, voltage_kv
    )
    return withdrawn_costs, injected_costs


In [8]:
from typing import Optional
from common.enums import ShiftType


def calculate_reactive_energy_costs(
    consumer_type: int,
    contracted_power_kW: float,
    active_kwh_in_f1f2f3: list,  
    withdrawn_kvarh_in_f1f2f3: list,     
    injected_kvarh_in_f1f2f3: list,
    voltage_kv: Optional[float]=None,
    threshold_withdrawn: float = 0.33
):
    total_reactive_energy_cost = 0.0
    withdrawn_costs, injected_costs = get_reactive_cost(
        consumer_type, contracted_power_kW, voltage_kv
    )
    for i in range(3): 
        threshold = active_kwh_in_f1f2f3[i] * threshold_withdrawn
        if withdrawn_kvarh_in_f1f2f3[i] > threshold:
            billable = withdrawn_kvarh_in_f1f2f3[i] - threshold
            total_reactive_energy_cost += billable * withdrawn_costs[i]

    for i in range(3):
        total_reactive_energy_cost += injected_kvarh_in_f1f2f3[i] * injected_costs[i]
    return total_reactive_energy_cost


In [9]:
def get_oneri_di_sistema(consumer_type, contracted_power_kW: float=None, voltage_kv: float=None):
    per_unit_costs_class_0 = per_kw_costs_class_0 = per_kwh_costs_class_0 = 0.0
    per_unit_costs_asos_1 = per_kw_costs_asos_1 = per_kwh_costs_asos_1 = 0.0
    per_unit_costs_asos_2 = per_kw_costs_asos_2 = per_kwh_costs_asos_2 = 0.0
    per_unit_costs_asos_3 = per_kw_costs_asos_3 = per_kwh_costs_asos_3 = 0.0
    per_unit_costs_val = per_kw_costs_val = per_kwh_costs_val = 0.0
    
    if consumer_type == 0 and contracted_power_kW <= 3: # BT domestico residente
        per_unit_costs_class_0, per_kw_costs_class_0, per_kwh_costs_class_0 = get_quote(
            tab_class0,
            consumer_type,
            contracted_power_kW,
            voltage_kv
        )
    elif consumer_type == 1 and contracted_power_kW <= 3: # BT domestico non residente
        per_unit_costs_asos_1, per_kw_costs_asos_1, per_kwh_costs_asos_1 = get_quote(
            tab_ASOS1,
            consumer_type,
            contracted_power_kW,
            voltage_kv
        )
    
    elif (
        (consumer_type in [2, 3, 4, 5]) or # BT non domestiche, illuminazione pubblica, ricarica EV
        (consumer_type == 0 and contracted_power_kW > 3) or  # BT domestico residente > 3 kW
        (consumer_type == 1 and contracted_power_kW > 3) or  # BT domestico non residente > 3 kW
        (consumer_type == 6 and contracted_power_kW <= 100) 
    ):
        per_unit_costs_asos_2, per_kw_costs_asos_2, per_kwh_costs_asos_2 = get_quote(
            tab_ASOS2,
            consumer_type,
            contracted_power_kW,
            voltage_kv
        )
    elif (
        (consumer_type == 6 and contracted_power_kW > 100) or # Media tensione
        consumer_type == 7 or # Alta tensione
        (consumer_type == 8 and voltage_kv is not None and voltage_kv < 380)
    ): 
        per_unit_costs_asos_3, per_kw_costs_asos_3, per_kwh_costs_asos_3 = get_quote(
            tab_ASOS3,
            consumer_type,
            contracted_power_kW,
            voltage_kv
        )
    elif consumer_type == 8 and voltage_kv is not None and voltage_kv >= 380: # Altissima tensione >=380 kV
        per_unit_costs_val, per_kw_costs_val, per_kwh_costs_val = get_quote(
            tab_VAL,
            consumer_type,
            contracted_power_kW,
            voltage_kv
        )
    else:
        raise ValueError("Tipo di consumatore o parametri non validi")
    
    return (
        per_unit_costs_class_0, per_kw_costs_class_0, per_kwh_costs_class_0,
        per_unit_costs_asos_1, per_kw_costs_asos_1, per_kwh_costs_asos_1,
        per_unit_costs_asos_2, per_kw_costs_asos_2, per_kwh_costs_asos_2,
        per_unit_costs_asos_3, per_kw_costs_asos_3, per_kwh_costs_asos_3,
        per_unit_costs_val, per_kw_costs_val, per_kwh_costs_val,
    )

In [10]:
def calculate_energy_cost(
    consumption_kwh,
    energy_unit_price_kWh,
):
    '''Calcola il costo della materia energia'''
    if isinstance(consumption_kwh, (int, float)) and isinstance(energy_unit_price_kWh, (int, float)):
        return consumption_kwh * energy_unit_price_kWh
    if isinstance(consumption_kwh, list) and isinstance(energy_unit_price_kWh, list):
        if len(consumption_kwh) != len(energy_unit_price_kWh):
            raise ValueError("Le liste devono avere la stessa lunghezza")
        total_energy_cost = 0
        for i in range(len(consumption_kwh)):
            total_energy_cost += consumption_kwh[i] * energy_unit_price_kWh[i]
        return total_energy_cost

def calculate_other_cost(
    consumption_kwh: float,
    losses_kwh: float,
    energy_unit_price_kWh: float,
    marketing_cost_per_month: float,
    dispbt_cost_per_month: float,
    dispatching_price_kWh: float,
    capacity_market_price: float
):
    ''' Calcola il costo della somma di perdite, dispacciamento, capacity, marketing e dispbt '''
    
    losses_cost = losses_kwh * energy_unit_price_kWh

    total_kwh = consumption_kwh + losses_kwh
    dispatching_cost = total_kwh * dispatching_price_kWh
    capacity_cost = total_kwh * capacity_market_price

    total_other_energy_cost = losses_cost + dispatching_cost + capacity_cost + marketing_cost_per_month + dispbt_cost_per_month
    return total_other_energy_cost


In [11]:
def calculate_accise(consumer_type: int, consumption_kwh, power_kW: float=None):
    accise_totali = 0
    if consumer_type == 0:
        if power_kW and power_kW > 3:
            accise_totali = consumption_kwh * 0.0227
        else:
            if consumption_kwh <= 150:
                accise_totali = 0
            else:
                accise_totali = consumption_kwh * 0.0227
    
    elif consumer_type == 1:
        accise_totali = consumption_kwh * 0.0227
    
    else:
        if consumption_kwh <= 200000:
            accise_totali = consumption_kwh * 0.0125
        else:
            accise_totali = 200000 * 0.0125 + (consumption_kwh - 200000) * 0.0075
    
    return accise_totali

def calculate_total_bill(
    consumer_type: int, consumption_kwh, power_kW, 
    total_energy_cost, total_other_energy_cost, 
    system_charges, total_reactive_energy_costs=0.0
):
    '''Calcola la bolletta compresa di IVA'''
    
    iva = 0.10 if consumer_type in [0, 1] else 0.22
    accise = calculate_accise(consumer_type, consumption_kwh, power_kW)
    total_oneri = total_other_energy_cost + sum(system_charges) + accise + total_reactive_energy_costs 
    imponibile = total_energy_cost + total_oneri
    energy_plus_iva = imponibile * (1 + iva)
    return energy_plus_iva

# Try with real bill

### RESIDENTIAL DOMESTIC

In [12]:
consumer_type = 0
consumption_kWh = 137 
losses_kwh = 14 
contracted_power_kW = 3.0
voltage_kv = None

energy_unit_price_kWh = 0.134  # €/kWh
marketing_cost_per_month = 8.00  # €/mese

dispbt_cost_per_month = 0.102592  # €/mese
dispatching_price_kWh = 0.009800  # €/kWh
capacity_market_price = 0.004275  # €/kWh


In [13]:
# Oneri di sistema 
(
    per_unit_costs_class_0, per_kw_costs_class_0, per_kwh_costs_class_0, 
    per_unit_costs_asos_1, per_kw_costs_asos_1, per_kwh_costs_asos_1,
    per_unit_costs_asos_2, per_kw_costs_asos_2, per_kwh_costs_asos_2,
    per_unit_costs_asos_3, per_kw_costs_asos_3, per_kwh_costs_asos_3,
    per_unit_costs_val, per_kw_costs_val, per_kwh_costs_val
) = get_oneri_di_sistema(
    consumer_type=consumer_type, contracted_power_kW=contracted_power_kW, voltage_kv=voltage_kv
)

per_unit_costs_arim, per_kw_costs_arim, per_kwh_costs_arim = get_quote(
    tab_ARIM,
    consumer_type,
    contracted_power_kW,
    voltage_kv
)
per_unit_costs_uc6, per_kw_costs_uc6, per_kwh_costs_uc6 = get_quote(
    tab_UC6,
    consumer_type,
    contracted_power_kW,
    voltage_kv
)
per_unit_costs_uc3, per_kw_costs_uc3, per_kwh_costs_uc3 = get_quote(
    tab_UC3,
    consumer_type,
    contracted_power_kW,
    voltage_kv
)

In [14]:
from entities.electricity_bill_system_costs import ElectricityBillSystemCosts

my_bill = ElectricityBillSystemCosts(
    consumer_type=consumer_type,  # Domestico Residente
    consumption_kwh=consumption_kWh,
    contracted_power_kW=contracted_power_kW,
    sigma_1=comp_s1,
    sigma_2=comp_s2,
    sigma_3=comp_s3,
    per_unit_costs= 0,
    per_kw_costs = 0,
    per_kwh_costs = 0,
    per_unit_costs_class_0=per_unit_costs_class_0,
    per_kw_costs_class_0=per_kw_costs_class_0,
    per_kwh_costs_class_0=per_kwh_costs_class_0,
    per_unit_costs_asos_1 = per_unit_costs_asos_1,
    per_kw_costs_asos_1 = per_kw_costs_asos_1,
    per_kwh_costs_asos_1 = per_kwh_costs_asos_1,
    per_unit_costs_asos_2 = per_unit_costs_asos_2,
    per_kw_costs_asos_2 = per_kw_costs_asos_2,
    per_kwh_costs_asos_2 = per_kwh_costs_asos_2,
    per_unit_costs_asos_3 = per_unit_costs_asos_3,
    per_kw_costs_asos_3 = per_kw_costs_asos_3,
    per_kwh_costs_asos_3 = per_kwh_costs_asos_3,
    per_unit_costs_val = per_unit_costs_val,
    per_kw_costs_val = per_kw_costs_val,
    per_kwh_costs_val = per_kwh_costs_val,

    per_unit_costs_arim = per_unit_costs_arim,
    per_kw_costs_arim = per_kw_costs_arim,
    per_kwh_costs_arim = per_kwh_costs_arim,
    per_unit_costs_uc3 = per_unit_costs_uc3,
    per_kw_costs_uc3 = per_kw_costs_uc3,
    per_kwh_costs_uc3 = per_kwh_costs_uc3,
    per_unit_costs_uc6 = per_unit_costs_uc6,
    per_kw_costs_uc6 = per_kw_costs_uc6,
    per_kwh_costs_uc6 = per_kwh_costs_uc6,
)


c:\DEV\LAB\electrical-energy-billing\venv\lib\site-packages\pydantic\main.py:250: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [15]:
my_oneri_sistema = my_bill.calculate_electricity_bill_system_costs(
    consumer_type=consumer_type, consumption_kwh=consumption_kWh, contracted_power_kW=contracted_power_kW
)
energy_cost = calculate_energy_cost(consumption_kWh, energy_unit_price_kWh)
other_energy_cost = calculate_other_cost(
    consumption_kwh=consumption_kWh,
    losses_kwh=losses_kwh,
    energy_unit_price_kWh=energy_unit_price_kWh,
    marketing_cost_per_month=marketing_cost_per_month,
    dispbt_cost_per_month=dispbt_cost_per_month,
    dispatching_price_kWh=dispatching_price_kWh,
    capacity_market_price=capacity_market_price
)

canone_rai = 9.65
total_bill_to_pay = calculate_total_bill(consumer_type, consumption_kWh, contracted_power_kW, energy_cost, other_energy_cost, my_oneri_sistema)
print(f"Bill: {round(total_bill_to_pay + canone_rai, 2)}")

Bill: 58.96


### IMPRESA

In [26]:
consumer_type = 4
consumption_kWh = 1675 
contracted_power_kW = 14.7
power_kW = 35 
voltage_kv = None

energy_unit_price_kWh = 0.25841 
energy_unit_price_kWh_in_f1f2f3 = [0.25841, 0.25985, 0.2389]
marketing_cost_per_month = 12.0
dispatching_price_kWh = 0.021218

withdrawn_kvarh_in_f1f2f3 = [16, 8, 13] # prelievi = capacità + induttiva
injected_kvarh_in_f1f2f3 = [0, 0, 325] # immissioni
active_kWh_in_f1f2f3 = [1150, 225, 300]

capacity_market_price = 0 # ??
losses_kwh = 0 # ??
dispbt_cost_per_month = 0 # ??


In [27]:
# Fattori di perdita
per_unit_costs, per_kw_costs, per_kwh_costs = get_quote(
    tab3,
    consumer_type,
    power_kW,
    voltage_kv
)

In [33]:
# Oneri di sistema 
(
    per_unit_costs_class_0, per_kw_costs_class_0, per_kwh_costs_class_0, 
    per_unit_costs_asos_1, per_kw_costs_asos_1, per_kwh_costs_asos_1,
    per_unit_costs_asos_2, per_kw_costs_asos_2, per_kwh_costs_asos_2,
    per_unit_costs_asos_3, per_kw_costs_asos_3, per_kwh_costs_asos_3,
    per_unit_costs_val, per_kw_costs_val, per_kwh_costs_val
) = get_oneri_di_sistema(
    consumer_type=consumer_type, contracted_power_kW=power_kW, voltage_kv=voltage_kv
)

per_unit_costs_arim, per_kw_costs_arim, per_kwh_costs_arim = get_quote(
    tab_ARIM,
    consumer_type,
    power_kW,
    voltage_kv
)
per_unit_costs_uc6, per_kw_costs_uc6, per_kwh_costs_uc6 = get_quote(
    tab_UC6,
    consumer_type,
    power_kW,
    voltage_kv
)
per_unit_costs_uc3, per_kw_costs_uc3, per_kwh_costs_uc3 = get_quote(
    tab_UC3,
    consumer_type,
    power_kW,
    voltage_kv
)

In [34]:
per_unit_costs_uc6, per_kw_costs_uc6, per_kwh_costs_uc6

(np.float64(1.6824), np.float64(nan), np.float64(7e-05))

In [29]:
print(per_unit_costs_class_0, per_kw_costs_class_0, per_kwh_costs_class_0, 
    per_unit_costs_asos_1, per_kw_costs_asos_1, per_kwh_costs_asos_1,
    per_unit_costs_asos_2, per_kw_costs_asos_2, per_kwh_costs_asos_2,
    per_unit_costs_asos_3, per_kw_costs_asos_3, per_kwh_costs_asos_3,
    per_unit_costs_val, per_kw_costs_val, per_kwh_costs_val)

0.0 0.0 0.0 0.0 0.0 0.0 248.76 300.72 0.8532 0.0 0.0 0.0 0.0 0.0 0.0


In [30]:
from entities.electricity_bill_system_costs import ElectricityBillSystemCosts

enel_bill = ElectricityBillSystemCosts(
    consumer_type=consumer_type,  
    consumption_kwh=consumption_kWh,
    contracted_power_kW=contracted_power_kW,
    sigma_1=comp_s1,
    sigma_2=comp_s2,
    sigma_3=comp_s3,
    per_unit_costs=per_unit_costs,
    per_kw_costs=per_kw_costs,
    per_kwh_costs=per_kwh_costs,
    per_unit_costs_class_0=per_unit_costs_class_0,
    per_kw_costs_class_0=per_kw_costs_class_0,
    per_kwh_costs_class_0=per_kwh_costs_class_0,
    per_unit_costs_asos_1 = per_unit_costs_asos_1,
    per_kw_costs_asos_1 = per_kw_costs_asos_1,
    per_kwh_costs_asos_1 = per_kwh_costs_asos_1,
    per_unit_costs_asos_2 = per_unit_costs_asos_2,
    per_kw_costs_asos_2 = per_kw_costs_asos_2,
    per_kwh_costs_asos_2 = per_kwh_costs_asos_2,
    per_unit_costs_asos_3 = per_unit_costs_asos_3,
    per_kw_costs_asos_3 = per_kw_costs_asos_3,
    per_kwh_costs_asos_3 = per_kwh_costs_asos_3,
    per_unit_costs_val = per_unit_costs_val,
    per_kw_costs_val = per_kw_costs_val,
    per_kwh_costs_val = per_kwh_costs_val,

    per_unit_costs_arim = per_unit_costs_arim,
    per_kw_costs_arim = per_kw_costs_arim,
    per_kwh_costs_arim = per_kwh_costs_arim,
    per_unit_costs_uc3 = per_unit_costs_uc3,
    per_kw_costs_uc3 = per_kw_costs_uc3,
    per_kwh_costs_uc3 = per_kwh_costs_uc3,
    per_unit_costs_uc6 = per_unit_costs_uc6,
    per_kw_costs_uc6 = per_kw_costs_uc6,
    per_kwh_costs_uc6 = per_kwh_costs_uc6,
)


c:\DEV\LAB\electrical-energy-billing\venv\lib\site-packages\pydantic\main.py:250: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [31]:
enel_oneri_sistema = enel_bill.calculate_electricity_bill_system_costs(
    consumer_type=consumer_type, consumption_kwh=consumption_kWh, contracted_power_kW=power_kW
)

energy_cost = calculate_energy_cost(active_kWh_in_f1f2f3, energy_unit_price_kWh_in_f1f2f3)
other_energy_cost = calculate_other_cost(
    consumption_kwh=consumption_kWh,
    losses_kwh=losses_kwh,
    energy_unit_price_kWh=energy_unit_price_kWh,
    marketing_cost_per_month=marketing_cost_per_month,
    dispbt_cost_per_month=dispbt_cost_per_month,
    dispatching_price_kWh=dispatching_price_kWh,
    capacity_market_price=capacity_market_price
)

reactive_energy_costs = calculate_reactive_energy_costs(
    consumer_type=consumer_type, contracted_power_kW=power_kW, 
    active_kwh_in_f1f2f3=active_kWh_in_f1f2f3, 
    withdrawn_kvarh_in_f1f2f3=withdrawn_kvarh_in_f1f2f3, injected_kvarh_in_f1f2f3=injected_kvarh_in_f1f2f3
)

total_bill_enel = calculate_total_bill(
    consumer_type, consumption_kWh, power_kW, 
    energy_cost, other_energy_cost, enel_oneri_sistema, reactive_energy_costs
)
print(f"Bill: {round(total_bill_enel, 2)}")
print(f"energy: {energy_cost:.2f}, cost: {other_energy_cost:.2f}, oneri: {sum(enel_oneri_sistema):.2f}, reactive_cost: {reactive_energy_costs:.2f}")

Bill: 924.7
energy: 427.31, cost: 47.54, oneri: 258.77, reactive_cost: 3.40
